# Quantitative Momentum Startegy
Investing in the sotcks that have increase in price most

#   Library import

In [126]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

# import Our List stocks

In [127]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [128]:
from secrets import IEX_CLOUD_API_TOKEN

# Making our First call

In [129]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data  = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2941854353138,
 'week52high': 189.65,
 'week52low': 115.75,
 'week52highSplitAdjustOnly': 187.76,
 'week52lowSplitAdjustOnly': 121.39,
 'week52change': 0.2724631234933916,
 'sharesOutstanding': 16401406753,
 'float': 0,
 'avg10Volume': 92067462,
 'avg30Volume': 103804119,
 'day200MovingAvg': 159.43,
 'day50MovingAvg': 175.79,
 'employees': 150954,
 'ttmEPS': 6.31,
 'ttmDividendRate': 0.8898855611258404,
 'dividendYield': 0.005293224559312313,
 'nextDividendDate': '',
 'exDividendDate': '2022-01-28',
 'nextEarningsDate': '2022-04-20',
 'peRatio': 29.16845294770324,
 'beta': 1.3726929989140095,
 'maxChangePercent': 65.89034263704504,
 'year5ChangePercent': 4.682824931478863,
 'year2ChangePercent': 1.172698620000062,
 'year1ChangePercent': 0.2715055591051016,
 'ytdChangePercent': -0.028889615990023,
 'month6ChangePercent': 0.18420951528485,
 'month3ChangePercent': 0.14380196173832582,
 'month1ChangePercent': 0.003651809252100125,
 'day30ChangePer

# Parising our API Call

In [130]:
data['year1ChangePercent']

0.2715055591051016

# Batch API call

In [131]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_group = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))        

my_columns = ['Ticker','Price','One-Year Price Return','Number of Share to Buy']

In [54]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in (symbol_string.split(',')):
        try:
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['price'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ],index = my_columns),
                ignore_index=True
            )
        except:
            print("This Symbol having issue while fetching data->"+str(symbol))
final_dataframe

This Symbol having issue while fetching data->WLTW


,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,A,143.36,0.154629,N/A
1,AAL,17.10,-0.044791,N/A
2,AAP,237.81,0.474898,N/A
3,AAPL,176.89,0.277091,N/A
4,ABBV,145.93,0.362559,N/A
...,...,...,...,...
499,YUM,128.19,0.217306,N/A
500,ZBH,125.30,-0.230688,N/A
501,ZBRA,526.00,0.246208,N/A
502,ZION,72.52,0.545853,N/A


# Removing Low Momemtum stock

In [132]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True )
final_dataframe=  final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

C:\Users\aakash\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,0,275,LB,80.36,2.375547,2488
1,1,148,DVN,54.30,2.118189,3683
2,2,317,MRO,22.50,1.78459,8888
3,3,109,COP,93.08,1.187643,2148
4,4,344,NUE,116.49,1.158484,1716
5,5,160,EOG,116.50,1.144546,1716
6,6,175,FANG,138.31,1.112231,1446
7,7,299,MCHP,73.47,1.073055,2722
8,8,195,FTNT,314.52,1.05686,635
9,9,386,PXD,231.73,0.882716,863


# Calulating Number of Share to buy

In [133]:
portfoliosize = float(10000000)
postion_size = portfoliosize/len(final_dataframe)
postion_size
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Share to Buy'] = math.floor(postion_size/final_dataframe.loc[i,'Price'])
final_dataframe

C:\Users\aakash\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,0,275,LB,80.36,2.375547,2488
1,1,148,DVN,54.30,2.118189,3683
2,2,317,MRO,22.50,1.78459,8888
3,3,109,COP,93.08,1.187643,2148
4,4,344,NUE,116.49,1.158484,1716
5,5,160,EOG,116.50,1.144546,1716
6,6,175,FANG,138.31,1.112231,1446
7,7,299,MCHP,73.47,1.073055,2722
8,8,195,FTNT,314.52,1.05686,635
9,9,386,PXD,231.73,0.882716,863


# Build a Better Momemtum Startegy

Finding High Momemtum 

In [184]:
hqm_columns =[
    'Ticker',
    'Price',
    'Number Of Share to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',    
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
    
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
        batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        print(batch_api_url)
        data = requests.get(batch_api_url).json()
        for symbol in symbol_string.split(','):
            try:
                hqm_dataframe = hqm_dataframe.append(
                    pd.Series([
                        symbol,
                        float(data[symbol]['price']),
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',    
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A',
                        'N/A'
                    ],index=hqm_columns                   
                    ),ignore_index=True
                )
            except:
                print("This Symbol having issue while fetching data->"+str(symbol))
hqm_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,fb&types=price,stats&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT

,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.870,N/A,0.157306,N/A,-0.094053,N/A,-0.098593,N/A,-0.053778,N/A,N/A
1,AAL,18.180,N/A,-0.044341,N/A,-0.224009,N/A,-0.257379,N/A,-0.119558,N/A,N/A
2,AAP,226.860,N/A,0.470495,N/A,0.114204,N/A,-0.007778,N/A,-0.061751,N/A,N/A
3,AAPL,176.400,N/A,0.279484,N/A,0.189069,N/A,0.143243,N/A,0.003594,N/A,N/A
4,ABBV,146.330,N/A,0.356301,N/A,0.2682,N/A,0.21979,N/A,0.052529,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,132.350,N/A,0.217969,N/A,-0.058419,N/A,-0.007535,N/A,-0.087242,N/A,N/A
500,ZBH,116.954,N/A,-0.234549,N/A,-0.180057,N/A,-0.137951,N/A,-0.049435,N/A,N/A
501,ZBRA,510.060,N/A,0.238904,N/A,-0.108972,N/A,-0.176726,N/A,-0.096553,N/A,N/A
502,ZION,71.690,N/A,0.547192,N/A,0.3114,N/A,0.108503,N/A,0.033958,N/A,N/A


# Calculating Momemtum Percentile

In [185]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
hqm_dataframe
list1 = []
for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        change_col = f'{time_period} Price Return'
#          print(hqm_dataframe[change_col])
#          print(hqm_dataframe.loc[row,change_col])
        hqm_dataframe.loc[row,percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],hqm_dataframe.loc[row,change_col])

hqm_dataframe



,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.870,N/A,0.157306,48.214286,-0.094053,23.214286,-0.098593,25.396825,-0.053778,42.65873,N/A
1,AAL,18.180,N/A,-0.044341,15.47619,-0.224009,4.761905,-0.257379,2.97619,-0.119558,12.5,N/A
2,AAP,226.860,N/A,0.470495,86.309524,0.114204,75.793651,-0.007778,55.15873,-0.061751,39.68254,N/A
3,AAPL,176.400,N/A,0.279484,67.063492,0.189069,85.515873,0.143243,91.865079,0.003594,72.619048,N/A
4,ABBV,146.330,N/A,0.356301,78.373016,0.2682,92.460317,0.21979,97.222222,0.052529,88.888889,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,132.350,N/A,0.217969,57.142857,-0.058419,30.753968,-0.007535,55.357143,-0.087242,25.0,N/A
500,ZBH,116.954,N/A,-0.234549,3.174603,-0.180057,9.722222,-0.137951,14.880952,-0.049435,44.642857,N/A
501,ZBRA,510.060,N/A,0.238904,60.31746,-0.108972,20.238095,-0.176726,9.722222,-0.096553,20.238095,N/A
502,ZION,71.690,N/A,0.547192,92.460317,0.3114,94.444444,0.108503,86.111111,0.033958,83.134921,N/A


# Calcuating HQM Score

HQM Score is mean of all Percentile 

Percentile --> How many stocks has price return less that price return

Example

0	A	143.870	N/A	0.157306	48.214286	
1	AAL	18.180	N/A	-0.044341	15.47619	
For AAL  has 15 Company has less than -0.44 price return

In [194]:
from statistics import mean

for row in hqm_dataframe.index[:1]:
    momemtum_percentile = []
    for time_period in time_periods:
        momemtum_percentile.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momemtum_percentile)
    
hqm_dataframe

,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.870,N/A,0.157306,48.214286,-0.094053,23.214286,-0.098593,25.396825,-0.053778,42.65873,34.871032
1,AAL,18.180,N/A,-0.044341,15.47619,-0.224009,4.761905,-0.257379,2.97619,-0.119558,12.5,8.928571
2,AAP,226.860,N/A,0.470495,86.309524,0.114204,75.793651,-0.007778,55.15873,-0.061751,39.68254,64.236111
3,AAPL,176.400,N/A,0.279484,67.063492,0.189069,85.515873,0.143243,91.865079,0.003594,72.619048,79.265873
4,ABBV,146.330,N/A,0.356301,78.373016,0.2682,92.460317,0.21979,97.222222,0.052529,88.888889,89.236111
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,132.350,N/A,0.217969,57.142857,-0.058419,30.753968,-0.007535,55.357143,-0.087242,25.0,42.063492
500,ZBH,116.954,N/A,-0.234549,3.174603,-0.180057,9.722222,-0.137951,14.880952,-0.049435,44.642857,18.105159
501,ZBRA,510.060,N/A,0.238904,60.31746,-0.108972,20.238095,-0.176726,9.722222,-0.096553,20.238095,27.628968
502,ZION,71.690,N/A,0.547192,92.460317,0.3114,94.444444,0.108503,86.111111,0.033958,83.134921,89.037698


# Selecting 50 best Momentum Stocks

In [203]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,level_0,index,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,317,MRO,22.641,N/A,1.82309,99.603175,0.816227,99.206349,0.276253,99.206349,0.196333,98.611111,99.156746
1,1,160,EOG,119.730,N/A,1.152425,99.007937,0.688682,99.007937,0.237791,98.015873,0.206411,99.007937,98.759921
2,2,109,COP,94.030,N/A,1.174209,99.206349,0.666682,98.412698,0.223851,97.619048,0.17789,98.412698,98.412698
3,3,212,HAL,33.200,N/A,0.706786,95.833333,0.605104,98.214286,0.305412,99.801587,0.24931,99.603175,98.363095
4,4,357,OXY,41.243,N/A,0.822716,97.81746,0.547276,97.81746,0.230742,97.81746,0.253874,99.801587,98.313492
5,5,495,XOM,85.534,N/A,0.762905,96.626984,0.454947,97.619048,0.279654,99.603175,0.199599,98.809524,98.164683
6,6,148,DVN,54.210,N/A,2.103423,99.801587,1.069733,100.0,0.245148,98.214286,0.093732,94.642857,98.164683
7,7,386,PXD,236.840,N/A,0.888431,98.214286,0.566659,98.015873,0.258741,98.611111,0.151591,97.619048,98.115079
8,8,275,LB,83.760,N/A,2.325138,100.0,0.845667,99.404762,0.219819,97.420635,0.080502,92.857143,97.420635
9,9,42,APA,35.010,N/A,0.803107,97.420635,0.920292,99.603175,0.16418,94.246032,0.173206,97.81746,97.271825


# Calculating the number of Share to Buy